<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/testDissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install control

In [ ]:
import numpy as np
import math
import control

**Plant**

$X(k+1) = Ax(k) + Bu(k)$

$y(k) = Cx(k)$

**Augmented state space**

$$\begin{bmatrix}
\Delta x(k+1) \\\\ y(k+1)
\end {bmatrix}
=
\begin{bmatrix}
A_p & 0_{s*s}\\\\ C_p * A_p & I
\end {bmatrix}
\begin{bmatrix}
\Delta x(k) \\\\ y(k)
\end {bmatrix}
+
\begin{bmatrix}
B_p \\\\ C_p * B_p
\end {bmatrix}
\Delta u(k)$$

#form plant model

In [56]:

class Modeling:
  def __init__(self, H1 = 10, H2 = 12, k1 = 4.3, k2 = 4.7, k3 = 24, area = 32, Np = 10, Nc = 4):
    self.H1 = H1
    self.H2 = H2
    self.k1 = k1
    self.k2 = k2
    self.k3 = k3
    self.area = area
    self.Np = Np
    self.Nc = Nc
    a11 = (-1 / self.area) * ((self.k1 / math.sqrt(self.H1)) + (self.k3 / math.sqrt(abs(self.H1 - self.H2))))
    a12 = (self.k3/ (2*self.area)) * (1 / math.sqrt(abs(self.H1 - self.H2)))
    a21 = (self.k3/ (2*self.area)) * (1 / math.sqrt(abs(self.H1 - self.H2)))
    a22 = (-1 / self.area) * ((self.k2 / math.sqrt(self.H2)) + (self.k3 / math.sqrt(abs(self.H1 - self.H2))))

    self.A_p = np.array([[a11, a12],
                [a21, a22]
                ])
    self.B_p = np.array([[1/self.area, 0],
                [0, 1/self.area]])
    self.C_p = np.eye(2, 2)
    self.D_p = np.zeros((2, 2))

  def discrete(self, Ts = 0.1, method = 'zoh'):
    Ts = 0.1
    # Create a state-space system
    sys_continuous = control.ss(self.A_p, self.B_p, self.C_p, self.D_p)

    # Discretize the system
    sys_discrete = sys_continuous.sample(Ts, method=method)

    Ad = sys_discrete.A
    Bd = sys_discrete.B
    Cd = sys_discrete.C
    Dd = sys_discrete.D
    return Ad, Bd, Cd, Dd

  def augment(self):
    Ad, Bd, Cd, Dd = self.discrete()
    states, _ = Ad.shape
    _, inputs = Bd.shape
    outputs, _ = Cd.shape

    A1 = np.hstack((Ad, np.zeros((states, outputs))))
    A2 = np.hstack((Cd @ Ad, np.eye(outputs, outputs)))
    self.Ae = np.vstack((A1, A2))
    self.Be = np.vstack((Bd,Cd @ Bd))
    self.Ce = np.hstack((np.zeros((outputs, outputs)), np.eye(outputs, outputs)))
    return self.Ae, self.Be, self.Ce

  def mpc_matrices(self):
    Ae, Be, Ce = self.augment()
    F = np.empty((Ce @ Ae).shape)
    Phi = []
    phi_dimx, phi_dimy = (Ce @ Ae @ Be).shape

    for i in range(1, self.Np+1):
      F = np.vstack((F, Ce @ (np.linalg.matrix_power(Ae, i))))
      row = []
      for j in range(self.Nc):
        Apower = i - j
        if Apower < 0:
          row.append(np.zeros((phi_dimx, phi_dimy)).tolist())
        else:
          row.append(((Ce @ np.linalg.matrix_power(Ae, Apower)) @ Be).tolist())
      row1 = []
      row2 = []
      for r in row:
        row1.extend(r[0])
        row2.extend(r[1])
      Phi.append(row1)
      Phi.append(row2)
    print("INDICATE PHI")
    print()
    Phi = np.array(Phi)

    F = F[2:, :]
    print("shape of F : ", F.shape)

    # Phi.T @ Phi
    x_ki = [0, 0, 1, 1]
    x_ki = np.array(x_ki)
    x_ki = x_ki.T
    Ref = [self.H1, self.H2]
    Ref = np.array([Ref[0] if (i % 2 == 0) else Ref[1] for i in range(self.Np * 2)])
    M1 = Phi.T @ Phi
    M2 = Phi.T @ F
    M3 = Phi.T @ Ref
    Rbar = 0.5
    DU = np.linalg.inv(M1 + (Rbar * np.eye(2*self.Nc))) @ (M3 - (M2 @ x_ki))
    list_version = DU.tolist()
    print("Control trajectory: ")
    print("_"*200)
    print(list_version)
plant = Modeling(Np = 20, Nc = 10, H1 = 15, H2 = 18)
disc = plant.mpc_matrices()
# print(disc)

INDICATE PHI

shape of F :  (40, 4)
Control trajectory: 
________________________________________________________________________________________________________________________________________________________________________________________________________
[14.120932814308585, 16.58157729057014, 12.915792718681718, 15.200029372776086, 11.683998255474544, 13.779345621784305, 10.512755387879464, 12.42511672554883, 9.402731187089623, 11.138302712271711, 8.354413481852438, 9.919675971797911, 7.368132270604806, 8.76984745411185, 6.444079836048119, 7.689291426858384, 5.582329559008505, 6.678368804970327, 4.782853430554626, 5.737349066996263]


In [36]:
1 - 7.639755494892597e-11

0.9999999999236024